In [1]:
"""
Install the Google AI Python SDK

$ pip install google-generativeai
"""

'\nInstall the Google AI Python SDK\n\n$ pip install google-generativeai\n'

In [ ]:
%pip install google-generativeai

In [3]:
def install_requirements():
  %pip install -r requirements.txt

def show_requirements():
  %pip list

def read_requirements():
  with open('requirements.txt', 'r') as file:
    lines = file.readlines()
    for line in lines:
        print(line.strip())

def pip_freeze_requirements():
  !pip freeze > requirements.txt

In [5]:
import os
import json
from dotenv import load_dotenv
import google.generativeai as genai

# Load environment variables from .env file
load_dotenv()

# Access variables
google_api_key = os.getenv('GOOGLE_API_KEY')
gemini_api_key = os.getenv('GEMINI_API_KEY')

In [6]:
# from google.colab import userdata

# API_KEY=userdata.get('GEMINI_API_KEY')
# API_KEY=os.environ["GEMINI_API_KEY"]
API_KEY=gemini_api_key

In [ ]:
API_KEY

In [8]:
genai.configure(api_key=API_KEY)

In [9]:
# Create the model
generation_config = {
  "temperature": 0.75,
  "top_p": 0.95,
  "top_k": 64,
  "max_output_tokens": 8192,
  "response_mime_type": "text/plain",
}

model = genai.GenerativeModel(
  model_name="gemini-1.5-pro-exp-0801",
  generation_config=generation_config,
  # safety_settings = Adjust safety settings
  # See https://ai.google.dev/gemini-api/docs/safety-settings
)


In [10]:
history=[
    {
        "role": "user",
        "parts": ["# HUMAN NATURAL LANGUAGE INTERPRETER TRANSPILES TO SQL QUERY (MYSQL) \n\nBy now, you are a sql (MySql) interpreter, where the user can \"query\" on database by asking as a natural language.\nYour response should be only the query between \"```SQL\" and \"```\".\n\n> Key Takeaway: This example demonstrates the successful translation of a natural language request into a structured SQL query, highlighting the importance of understanding the nuances of both human language and database querying.\n> This ability is crucial for bridging the gap between user needs and data retrieval.\n\nFor the context, follow the `users` and `orders` table structures:\n\n**`users` Table:**\n\n| Column Name      | Data Type      | Constraints                               | Description                                      |\n|------------------|-----------------|-------------------------------------------|--------------------------------------------------|\n| `id`             | INT UNSIGNED    | PRIMARY KEY, AUTO_INCREMENT                 | Unique identifier for each user                  |\n| `first_name`     | VARCHAR(255)   | NOT NULL                                  | User's first name                               |\n| `last_name`      | VARCHAR(255)   | NOT NULL                                  | User's last name                                |\n| `email`          | VARCHAR(255)   | NOT NULL, UNIQUE                           | User's email address (must be unique)           |\n| `active_status` | BOOLEAN        | DEFAULT TRUE                               | Indicates if the user is active (true/false)      |\n| `created_at`    | TIMESTAMP      | DEFAULT CURRENT_TIMESTAMP                 | Timestamp indicating when the user was created    |\n| `updated_at`    | TIMESTAMP      | DEFAULT CURRENT_TIMESTAMP ON UPDATE CURRENT_TIMESTAMP | Timestamp indicating when the user was last updated |\n\n**`orders` Table:**\n\n| Column Name      | Data Type      | Constraints                               | Description                                       |\n|------------------|-----------------|-------------------------------------------|---------------------------------------------------|\n| `id`             | INT UNSIGNED    | PRIMARY KEY, AUTO_INCREMENT                 | Unique identifier for each order                   |\n| `user_id`        | INT UNSIGNED    | NOT NULL, FOREIGN KEY (`user_id`) REFERENCES `users`(`id`) | Identifier of the user who placed the order        |\n| `order_date`     | DATETIME        | NOT NULL                                  | Date and time when the order was placed          |\n| `total_amount`   | DECIMAL(10, 2) | NOT NULL                                  | Total amount of the order (in currency units)     |\n| `created_at`    | TIMESTAMP      | DEFAULT CURRENT_TIMESTAMP                 | Timestamp indicating when the order was created     |\n| `updated_at`    | TIMESTAMP      | DEFAULT CURRENT_TIMESTAMP ON UPDATE CURRENT_TIMESTAMP | Timestamp indicating when the order was last updated  |\n\n**Key Relationships:**\n\n* **One-to-Many:** The `users` table has a one-to-many relationship with the `orders` table. One user can have multiple orders, but each order belongs to only one user. This relationship is established through the `user_id` foreign key in the `orders` table.\n\n**Data Types and Constraints:**\n\n* **`INT UNSIGNED`:** Integer values (positive whole numbers)\n* **`VARCHAR(255)`:** Variable-length string (up to 255 characters)\n* **`BOOLEAN`:** True/false values\n* **`TIMESTAMP`:** Stores date and time information\n* **`DECIMAL(10, 2)`:** Decimal values with a precision of 10 digits and a scale of 2 (e.g., 12345678.90)\n* **`PRIMARY KEY`:** Uniquely identifies each row in a table\n* **`AUTO_INCREMENT`:** Automatically generates the next sequential value for the primary key\n* **`NOT NULL`:** Ensures that a column cannot have a null value\n* **`UNIQUE`:** Ensures that all values in a column are unique\n* **`FOREIGN KEY`:** Establishes a link between two tables based on a common column (enforces referential integrity)\n\n\nUser request:\n\nEloquent.query(\"I need a list of all active users who have spent more than $100 in the last month. The list should include their name, email, total amount spent, average order value, their latest order date, and how many orders they've placed. Sort the list by the total amount spent, with the highest spenders at the top.\");\n\nAssistant answer: \n\n```SQL\n-- Retrieve active users who have placed orders with a total value exceeding $100 in the last month,\n-- along with their average order value, latest order date, and number of orders.\n\nSELECT \n    u.id AS user_id,                  -- User ID\n    u.first_name,                     -- User's first name\n    u.last_name,                      -- User's last name\n    u.email,                          -- User's email address\n    SUM(o.total_amount) AS total_spent, -- Total amount spent by the user\n    AVG(o.total_amount) AS average_order_value, -- Average order value\n    MAX(o.order_date) AS latest_order_date, -- Date of the latest order\n    COUNT(o.id) AS total_orders         -- Total number of orders placed\nFROM \n    users u                           -- From the users table (aliased as 'u')\nINNER JOIN \n    orders o ON u.id = o.user_id     -- Join with the orders table (aliased as 'o') based on matching user IDs\nWHERE \n    u.active_status = 1             -- Filter for active users\n    AND o.order_date >= DATE_SUB(CURDATE(), INTERVAL 1 MONTH) -- Filter for orders within the last month\nGROUP BY \n    u.id                             -- Group the results by user ID\nHAVING \n    total_spent > 100                -- Filter for users with total spending exceeding $100\nORDER BY \n    total_spent DESC;                -- Order the results by total spending in descending order;\n```\n"]
    },
    {
        "role": "model",
        "parts": ["```sql\nSELECT\n    u.first_name,\n    u.last_name,\n    u.email,\n    SUM(o.total_amount) AS total_spent,\n    AVG(o.total_amount) AS average_order_value,\n    MAX(o.order_date) AS latest_order_date,\n    COUNT(o.id) AS total_orders\nFROM\n    users u\nJOIN\n    orders o ON u.id = o.user_id\nWHERE\n    u.active_status = TRUE\n    AND o.order_date >= DATE_SUB(CURDATE(), INTERVAL 1 MONTH)\nGROUP BY\n    u.id, u.first_name, u.last_name, u.email\nHAVING\n    total_spent > 100\nORDER BY\n    total_spent DESC;\n```"]
    },
    {
        "role": "user",
        "parts": ["Eloquent.query(\"select all users where the user `active_status` is true/not null/greater than zero and join users on orders table, where orders table has `user_id` field to match his orders and these orders need to be ordered by the orders.id as DESC\");\n"]
    },
    {
        "role": "model",
        "parts": ["```sql\nSELECT\n    u.*\nFROM\n    users u\nJOIN\n    orders o ON u.id = o.user_id\nWHERE\n    u.active_status = TRUE\nORDER BY\n    o.id DESC;\n```\n"]
    }
]

In [11]:
chat_session = model.start_chat(history=history)

In [12]:
def reset_chat_session():
  chat_session = model.start_chat(history=history)
  return chat_session

In [13]:
def print_out_chat_history(chat_session):
  ch = chat_session.history
  for i in ch:
    print("\n", '='*78, "\n")
    ch_role = i.role
    ch_parts = '\n'.join([p.text for p in i.parts]) # Call join on the delimiter string
    print(ch_role, ch_parts)
    print("\n", '-'*78, "\n")

print_out_chat_history(chat_session)



user # HUMAN NATURAL LANGUAGE INTERPRETER TRANSPILES TO SQL QUERY (MYSQL) 

By now, you are a sql (MySql) interpreter, where the user can "query" on database by asking as a natural language.
Your response should be only the query between "```SQL" and "```".

> Key Takeaway: This example demonstrates the successful translation of a natural language request into a structured SQL query, highlighting the importance of understanding the nuances of both human language and database querying.
> This ability is crucial for bridging the gap between user needs and data retrieval.

For the context, follow the `users` and `orders` table structures:

**`users` Table:**

| Column Name      | Data Type      | Constraints                               | Description                                      |
|------------------|-----------------|-------------------------------------------|--------------------------------------------------|
| `id`             | INT UNSIGNED    | PRIMARY KEY, AUTO_INCREMENT

In [14]:
def get_user_input_templated(user_input: str = ""):
  formatted_string = f'User: Enloquent.query("{user_input}")' # Use {} as placeholder
  input_data = {
      "user_input": formatted_string # Store the formatted string in the dictionary
  }
  # JSON stringify
  input_data = json.dumps([input_data["user_input"]], indent=2)
  return json.loads(input_data)[0]

In [15]:
user_prompt = """select all users where the user active_status is true/not null/greater than zero and
join users on orders table, where orders table has user_id field to match his orders and
these orders need to be ordered by the orders.id as DESC"""
user_prompt

'select all users where the user active_status is true/not null/greater than zero and\njoin users on orders table, where orders table has user_id field to match his orders and\nthese orders need to be ordered by the orders.id as DESC'

In [16]:
templated_prompt_1 = get_user_input_templated(user_prompt)
print(templated_prompt_1)
templated_prompt_2 = get_user_input_templated('Select the best sellers products ordered by category most sell followd by product most sell')
print(templated_prompt_2)

User: Enloquent.query("select all users where the user active_status is true/not null/greater than zero and
join users on orders table, where orders table has user_id field to match his orders and
these orders need to be ordered by the orders.id as DESC")
User: Enloquent.query("Select the best sellers products ordered by category most sell followd by product most sell")


In [17]:
def send_to_chat(prompt: str = ''):
  try:
    response = chat_session.send_message(prompt)
    return response.text
  except Exception as e:
    print(f"An error occurred: {e}")
    return None

In [21]:
# @title Recursive UX V-0.3

def Ux():
  user_input = input("User natual language prompt to SQL query (MySQL): ")
  prompt = get_user_input_templated(user_input)
  print(prompt)
  confirm = input("Confirm? (y/n): ")
  if confirm.lower() == "n":
    return Ux()
  try:
    body = send_to_chat(prompt)
  except Exception as e:
    print(f"An error occurred: {e}")
    return Ux()

  quite = input("Wish to quite? (y/n): ")
  if quite.lower() == "y":
    reset_chat = input("Reset chat session? (y/n): ")
    if reset_chat.lower() == "y":
      reset_chat_session()
    return [{'query': body, 'user_input': user_input}]
  return [{'query': body, 'user_input': user_input}] + Ux()

sql_query = Ux()
if sql_query:
  print(json.dumps(sql_query, indent=2))

# select orders ordering by id as DESC pagination for 20 orders per page
# select all user that have at least 1 order this month
# select all users that have spent more than $100 in the last month. The list should include their name, email, total amount spent, average order value, their latest order date, and how many orders they've placed. Sort the list by the total amount spent, with the highest spenders at the top.


User: Enloquent.query("select all users that have spent more than $100 in the last month. The list should include their name, email, total amount spent, average order value, their latest order date, and how many orders they've placed. Sort the list by the total amount spent, with the highest spenders at the top.")
[
  {
    "query": "```sql\nSELECT\n    u.first_name,\n    u.last_name,\n    u.email,\n    SUM(o.total_amount) AS total_spent,\n    AVG(o.total_amount) AS average_order_value,\n    MAX(o.order_date) AS latest_order_date,\n    COUNT(o.id) AS total_orders\nFROM\n    users u\nJOIN\n    orders o ON u.id = o.user_id\nWHERE\n    o.order_date >= DATE_SUB(CURDATE(), INTERVAL 1 MONTH)\nGROUP BY\n    u.id, u.first_name, u.last_name, u.email\nHAVING\n    total_spent > 100\nORDER BY\n    total_spent DESC;\n```",
    "user_input": "select all users that have spent more than $100 in the last month. The list should include their name, email, total amount spent, average order value, their la

In [24]:
for query in sql_query:
  # print user input with auto wrap word
  print('='*80,"\n"*2)
  print(query["user_input"])
  # print query breaking line
  print('-'*5,"\n"*2,query["query"])



select all users that have spent more than $100 in the last month. The list should include their name, email, total amount spent, average order value, their latest order date, and how many orders they've placed. Sort the list by the total amount spent, with the highest spenders at the top.
----- 

 ```sql
SELECT
    u.first_name,
    u.last_name,
    u.email,
    SUM(o.total_amount) AS total_spent,
    AVG(o.total_amount) AS average_order_value,
    MAX(o.order_date) AS latest_order_date,
    COUNT(o.id) AS total_orders
FROM
    users u
JOIN
    orders o ON u.id = o.user_id
WHERE
    o.order_date >= DATE_SUB(CURDATE(), INTERVAL 1 MONTH)
GROUP BY
    u.id, u.first_name, u.last_name, u.email
HAVING
    total_spent > 100
ORDER BY
    total_spent DESC;
```
